In [21]:
import urllib.request
import urllib.parse
import http.cookiejar
import openpyxl
import json
import datetime
import io
from bs4 import BeautifulSoup

In [22]:
cookie_handler = urllib.request.HTTPCookieProcessor()
opener = urllib.request.build_opener(cookie_handler)
response = opener.open("https://keycloak-transparency.entsoe.eu/realms/tp/protocol/openid-connect/auth?response_type=code&client_id=tp-web&redirect_uri=https%3A%2F%2Ftransparency.entsoe.eu%2Fsso%2Flogin&state=706544b9-1294-4824-acaa-12d5935d17a8&login=true&scope=openid")
response_html = BeautifulSoup(response, "html.parser")

In [24]:
login_url = response_html.find("form")["action"]
with open("entso-credentials.json") as fh:
    credentials = json.load(fh)

form_fields = {
    "username": credentials["username"],
    "password": credentials["password"],
    "credentialId": ""
}
form_data = urllib.parse.urlencode(form_fields).encode("utf-8")
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}
login_request = urllib.request.Request(login_url, method="POST", data=form_data, headers=headers)

In [25]:
try:
    response = opener.open(login_request)
except urllib.request.HTTPError as error:
    if not error.url.startswith("https://transparency.entsoe.eu/sso/login"):
        raise

In [26]:
def download_date(opener, dt):
    date_str = dt.strftime("%d.%m.%Y")
    xlsx_url = f"https://transparency.entsoe.eu/generation/r2/dayAheadGenerationForecastWindAndSolar/export?name=&defaultValue=true&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime={date_str}+00%3A00%7CCET%7CDAYTIMERANGE&dateTime.endDateTime={date_str}+00%3A00%7CCET%7CDAYTIMERANGE&area.values=CTY%7C10YES-REE------0!BZN%7C10YES-REE------0&productionType.values=B16&productionType.values=B18&productionType.values=B19&processType.values=A18&processType.values=A01&processType.values=A40&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=DEFAULT&exportType=XLSX"
    with opener.open(xlsx_url) as fh:
        byte_data = fh.read()
    buf = io.BytesIO(byte_data)
    wb = openpyxl.open(buf)
    return wb

In [27]:
download_date(opener, dt)

In [29]:
list(wb.active.values)[:10]

[('Generation Forecasts - Day Ahead for Wind and Solar',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('Day-ahead Generation Forecasts for Wind and Solar [14.1.D]',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('BZN|ES 01.06.2022 00:00 - 02.06.2022 00:00 (CET/CEST)',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None, None, None, None, None, None, None, None, None, None),
 ('MTU', 'BZN|ES', None, None, 'BZN|ES', None, None, 'BZN|ES', None, None),
 (None,
  'Generation Forecast',
  None,
  None,
  'Generation Forecast',
  None,
  None,
  'Generation Forecast',
  None,
  None),
 (None,
  'Solar',
  None,
  None,
  'Wind Offshore',
  None,
  None,
  'Wind Onshore',
  None,
  None),
 (None,
  'Day Ahead',
  'Intraday',
  'Current',
  'Day Ahead',
  'Intraday',
  'Current',
  'Day Ahead',
  'Intraday',
  'Current'),
 ('00:00 - 00:15',
  '443',
  '633',
  '633',
  'n/e',
  'n/e',
  'n/e',
  '3504',
  '3051